<a href="https://colab.research.google.com/github/Chetna-Jha-QC/IBMSummerSchool1/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit[visualization]==1.1.0
# Use the following if you are on MacOS/zsh
#!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
!pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 5.3 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=d06883a66798427a0d5d4c0f8b4c8d98099fee12f3384289f2ac12533e627aed
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%set_env QXToken=86b6b9e5a248f8ae53d07dbf584a45208b12702ed9fe50ce1072f368e52630cae7b9b45548fdaedbbf158b4587b0a89551bc42c01719da57e021e472b14c17c7

env: QXToken=86b6b9e5a248f8ae53d07dbf584a45208b12702ed9fe50ce1072f368e52630cae7b9b45548fdaedbbf158b4587b0a89551bc42c01719da57e021e472b14c17c7


In [ ]:
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [ ]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [ ]:
def get_qc_characteristics(qc):

    depth = qc.depth()
    num_qubits = qc.num_qubits
    ops = [op.name for op, _, _ in qc.data]
    num_multi_qubit_ops = sum(1 for op, _, _ in qc.data if len(op.qubits) > 1)

    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [ ]:
def get_qc_characteristics(qc):
    # Determine the number of qubits in the circuit
    num_qubits = qc.num_qubits

    # Determine the operations in the circuit
    ops = {}
    for gate, qargs, _ in qc.data:
        gate_name = gate.name
        if gate_name in ops:
            ops[gate_name] += 1
        else:
            ops[gate_name] = 1

    # Determine the depth of the circuit
    depth = qc.depth()

    # Determine the number of multi-qubit operations
    num_multi_qubit_ops = sum(1 for gate, qargs, _ in qc.data if len(qargs) > 1)

    # Return the characteristics as a dictionary
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [ ]:
# Grade your work!
grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [ ]:


#end of lab of ex1

In [ ]:
def get_qc_characteristics(qc):

    depth = qc.depth()
    num_qubits = qc.num_qubits
    ops = [op.name for op, _, _ in qc.data]
    num_multi_qubit_ops = sum(1 for op, _, _ in qc.data if len(op.qubits) > 1)

    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}







def get_qc_characteristics(qc):
    # Determine the number of qubits in the circuit
    num_qubits = qc.num_qubits

    # Determine the operations in the circuit
    ops = {}
    for gate, qargs, _ in qc.data:
        gate_name = gate.name
        if gate_name in ops:
            ops[gate_name] += 1
        else:
            ops[gate_name] = 1

    # Determine the depth of the circuit
    depth = qc.depth()

    # Determine the number of multi-qubit operations
    num_multi_qubit_ops = sum(1 for gate, qargs, _ in qc.data if len(qargs) > 1)

    # Return the characteristics as a dictionary
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)
#
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [ ]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [ ]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                                                                                                                                                                   ┌───┐    ┌────────────┐   ┌───┐    ┌───────────┐                                                          ┌───┐    ┌────────────┐                         ┌───┐ ┌───────────┐               ┌───┐┌───────────┐                        ┌───┐ ┌──────────┐          ┌───┐     ┌───────────┐             ┌───┐  ┌──────────┐                                        ┌───┐    ┌───────────┐        ┌───┐     ┌──────────┐          ┌───┐     ┌──────────┐              ┌───┐ ┌─────────┐              ┌───┐┌──────────┐            ┌───┐ ┌──────────┐   ┌────┐   ┌──────────┐                                                                                                                                             
q_2 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/256) ├───┤ X ├────┤ Rz(π/256) ├──────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/128) ├─────────────────────────┤ X ├─┤ Rz(π/128) ├───────────────┤ X ├┤ Rz(-π/64) ├────────────────────────┤ X ├─┤ Rz(π/64) ├──────────┤ X ├─────┤ Rz(-π/32) ├─────────────┤ X ├──┤ Rz(π/32) ├────────────────────────────────────────┤ X ├────┤ Rz(-π/16) ├────────┤ X ├─────┤ Rz(π/16) ├──────────┤ X ├─────┤ Rz(-π/8) ├──────────────┤ X ├─┤ Rz(π/8) ├──────────────┤ X ├┤ Rz(-π/4) ├────────────┤ X ├─┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(7π/8) ├──────────────────■──────────────────────■────────────────■─────────────────■────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                   └─┬─┘    └────────────┘   └─┬─┘    └───────────┘                                                          └─┬─┘    └────────────┘                ┌───┐    └─┬─┘┌┴───────────┴┐    ┌───┐     └─┬─┘├───────────┴┐                  ┌───┐└─┬─┘┌┴──────────┴┐         └─┬─┘     └───┬───┬───┘┌───────────┐└─┬─┘  └──────────┘ ┌───┐┌────────────┐        ┌───┐       └─┬─┘    ├───────────┤┌───┐   └─┬─┘    ┌┴──────────┴┐┌───┐    └─┬─┘    ┌┴──────────┤    ┌───┐     └─┬─┘┌┴─────────┴┐    ┌───┐    └─┬─┘├──────────┤   ┌───┐    └─┬─┘┌┴──────────┤   ├───┬┘   ├──────────┤           ┌───┐  │  ┌───────────┐┌───┐  │  ┌──────────┐┌─┴─┐ ┌──────────┐┌─┴─┐┌─────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌────┐┌─────────┐
q_0 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────┼───────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────┤ X ├──────┼──┤ Rz(-π/1024) ├────┤ X ├───────┼──┤ Rz(π/1024) ├──────────────────┤ X ├──┼──┤ Rz(-π/512) ├───────────┼───────────┤ X ├────┤ Rz(π/512) ├──┼─────────────────┤ X ├┤ Rz(-π/256) ├────────┤ X ├─────────┼──────┤ Rz(π/256) ├┤ X ├─────┼──────┤ Rz(-π/128) ├┤ X ├──────┼──────┤ Rz(π/128) ├────┤ X ├───────┼──┤ Rz(-π/64) ├────┤ X ├──────┼──┤ Rz(π/64) ├───┤ X ├──────┼──┤ Rz(-π/32) ├───┤ X ├────┤ Rz(π/32) ├───────────┤ X ├──┼──┤ Rz(-π/16) ├┤ X ├──┼──┤ Rz(π/16) ├┤ X ├─┤ Rz(-π/8) ├┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
         ┌─────────┐┌────┐┌────────────┐

In [ ]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'sx': 10, 'cx': 90}
  Number of multi-qubit Operations: 90


In [ ]:
# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Transpile the circuit for the backend with the linear coupling map
preset_pm = generate_preset_pass_manager(
    2, basis_gates=backend.operation_names, coupling_map=cm
)
qc_routed = preset_pm.run(qc.decompose())

In [ ]:
# Grade your work
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


 pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names))

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.passes import BasicSwap

# ... (rest of your code)
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

print_qc_characteristics(qc_synth)

# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Transpile the circuit for the backend with the linear coupling map
preset_pm = generate_preset_pass_manager(
    2, basis_gates=backend.operation_names, coupling_map=cm
)
qc_routed = preset_pm.run(qc.decompose())

########################################################################################################
# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Staged Pass Manager for custom transpilation
pm_staged = StagedPassManager()

# Replace n-qubit QFT with decomposition (unchanged)
pm_staged.init = generate_unroll_3q(None)

# Layout: Trivial mapping (each qubit to corresponding physical qubit)
pm_staged.layout = PassManager()
pm_staged.layout += TrivialLayout(cm)
pm_staged.layout += generate_embed_passmanager(cm)

# Routing: BasicSwap to handle required swaps for linear connectivity
pm_staged.routing = PassManager()
pm_staged.routing += BasicSwap(coupling_map=cm)  # Insert swaps for compatible layout

# Transpile the circuit using your custom pass manager
qc_routed = pm_staged.run(qc.decompose())
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)


Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']
Quantum circuit characteristics
  Depth: 80
  Number of qubits: 12
  Operations: {'rz': 145, 'sx': 12, 'cx': 132, 'barrier': 1, 'measure': 12}
  Number of multi-qubit Operations: 133
Quantum circuit characteristics
  Depth: 155
  Number of qubits: 12
  Operations: {'h': 12, 'cp': 66, 'swap': 476, 'barrier': 1, 'measure': 12}
  Number of multi-qubit Operations: 543


┌───┐                                  ░                ┌─┐                  
  q_0 -> 0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────■──────────────■─────────────■─────────────■─────────────■────────────■────────────■────────────■───────────■───────────■───────┤ H ├─X─────X─────X─────X─────X─────X──░────────────────┤M├──────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     ┌───┐    │P(π/2048)     │P(π/1024)     │P(π/512)     │P(π/256)     │P(π/128)     │P(π/64)     │P(π/32)     │P(π/16)     │P(π/8)     │P(π/4)     │P(π/2) └───┘ │     │     │     │     │     │  ░                └╥┘┌─┐               
  q_1 -> 1 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────■─────────────■─────────────■─────────────■────────────■────────────■────────────■───────────■───────────■───────┤ H ├─X──■───────────X──■───────────X──■──────────X──■──────────X──■──────────X──■─────────X──■─────────X──■─────────X──■────────X──■────────X──■─────────────X──X──X──X──X──X──X──X──X──X──X──░─────────────────╫─┤M├───────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ┌───┐    │P(π/1024)     │P(π/512)     │P(π/256)     │P(π/128)     │P(π/64)     │P(π/32)     │P(π/16)     │P(π/8)     │P(π/4)     │P(π/2) └───┘ │              │              │             │             │             │            │            │            │           │           │                   │     │     │     │     │     ░            

In [ ]:
# grade your work!

grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Failed:  Not successful - 


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.passes import BasicSwap

# ... (rest of your code)
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

print_qc_characteristics(qc_synth)

# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Transpile the circuit for the backend with the linear coupling map
preset_pm = generate_preset_pass_manager(
    2, basis_gates=backend.operation_names, coupling_map=cm
)
qc_routed = preset_pm.run(qc.decompose())

########################################################################################################
# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor





pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)



pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names))
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)



Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']
Quantum circuit characteristics
  Depth: 80
  Number of qubits: 12
  Operations: {'rz': 145, 'sx': 12, 'cx': 132, 'barrier': 1, 'measure': 12}
  Number of multi-qubit Operations: 133
Quantum circuit characteristics
  Depth: 95
  Number of qubits: 12
  Operations: {'rz': 222, 'sx': 12, 'cx': 150, 'barrier': 1, 'measure': 12}
  Number of multi-qubit Operations: 151


global phase: 4.7128
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ┌───┐┌─────────────┐                         ┌───┐┌────────────┐                               ┌───┐┌─────────────┐                               ┌───┐┌────────────┐                    ┌───┐┌─────────────┐                    ┌───┐┌────────────┐                          ┌───┐┌────────────┐                          ┌───┐┌───────────┐               ┌───┐┌────────────┐               ┌───┐┌───────────┐                     ┌───┐┌────────────┐                     ┌───┐┌───────────┐          ┌───┐┌───────────┐          ┌───┐┌──────────┐                ┌───┐┌───────────┐                ┌───┐┌──────────┐     ┌───┐┌───────────┐     ┌───┐┌──────────┐           ┌───┐┌──────────┐           ┌───┐┌─────────┐┌───┐┌──────────┐┌───┐┌─────────┐┌─────────┐┌────┐┌─────────┐     ┌───┐      ░ ┌─┐                                 
  q_0 -> 0 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/4096) ├─────────────────────────┤ X ├┤ Rz(π/4096) ├───────────────────────────────┤ X ├┤ Rz(-π/2048) ├───────────────────────────────┤ X ├┤ Rz(π/2048) ├────────────────────┤ X ├┤ Rz(-π/1024) ├────────────────────┤ X ├┤ Rz(π/1024) ├──────────────────────────┤ X ├┤ Rz(-π/512) ├──────────────────────────┤ X ├┤ Rz(π/512) ├───────────────┤ X ├┤ Rz(-π/256) ├───────────────┤ X ├┤ Rz(π/256) ├─────────────────────┤ X ├┤ Rz(-π/128) ├─────────────────────┤ X ├┤ Rz(π/128) ├──────────┤ X ├┤ Rz(-π/64) ├──────────┤ X ├┤ Rz(π/64) ├────────────────┤ X ├┤ Rz(-π/32) ├────────────────┤ X ├┤ Rz(π/32) ├─────┤ X ├┤ Rz(-π/16) ├─────┤ X ├┤ Rz(π/16) ├───────────┤ X ├┤ Rz(-π/8) ├───────────┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ X ├──■───░─┤M├─────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐┌─────────────┐                               ┌───┐┌────────────┐                    ┌───┐└─┬─┘├─────────────┤                    ┌───┐└─┬─┘├────────────┤                          ┌───┐└─┬─┘└

In [ ]:
# grade your work!
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Failed:  Not successful - 


In [ ]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()
def get_qc_characteristics(qc):

    depth = qc.depth()
    num_qubits = qc.num_qubits
    ops = [op.name for op, _, _ in qc.data]
    num_multi_qubit_ops = sum(1 for op, _, _ in qc.data if len(op.qubits) > 1)

    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}







def get_qc_characteristics(qc):
    # Determine the number of qubits in the circuit
    num_qubits = qc.num_qubits

    # Determine the operations in the circuit
    ops = {}
    for gate, qargs, _ in qc.data:
        gate_name = gate.name
        if gate_name in ops:
            ops[gate_name] += 1
        else:
            ops[gate_name] = 1

    # Determine the depth of the circuit
    depth = qc.depth()

    # Determine the number of multi-qubit operations
    num_multi_qubit_ops = sum(1 for gate, qargs, _ in qc.data if len(qargs) > 1)

    # Return the characteristics as a dictionary
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)
#
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)
#
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)
#
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)
#

print_qc_characteristics(qc_synth)

#

# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Transpile the circuit for the backend with the linear coupling map
preset_pm = generate_preset_pass_manager(
    2, basis_gates=backend.operation_names, coupling_map=cm
)
qc_routed = preset_pm.run(qc.decompose())
# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor

# Transpile the circuit for the backend with the linear coupling map
preset_pm = generate_preset_pass_manager(
    2, basis_gates=backend.operation_names, coupling_map=cm
)
qc_routed = preset_pm.run(qc.decompose())

########################################################################################################
# Define a 10-qubit linear coupling map
cm = CouplingMap([[i, i + 1] for i in range(num_qubits - 1)])  # Connects each qubit to its neighbor





pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)



pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names))
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)


Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1
Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']
Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'sx': 10, 'cx': 90}
  Number of multi-qubit Operations: 90
Quantum circuit characteristics
  Depth: 75
  Number of qubits: 10
  Operations: {'rz': 155, 'sx': 10, 'cx': 90}
  Number of multi-qubit Operations: 90


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ┌───┐┌─────────────┐                    ┌───┐┌────────────┐                          ┌───┐┌────────────┐                          ┌───┐┌───────────┐               ┌───┐┌────────────┐               ┌───┐┌───────────┐                     ┌───┐┌────────────┐                     ┌───┐┌───────────┐          ┌───┐┌───────────┐          ┌───┐┌──────────┐                ┌───┐┌───────────┐                ┌───┐┌──────────┐     ┌───┐┌───────────┐     ┌───┐┌──────────┐           ┌───┐┌──────────┐           ┌───┐┌─────────┐┌───┐┌──────────┐┌───┐┌─────────┐┌─────────┐┌────┐┌─────────┐
q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/1024) ├────────────────────┤ X ├┤ Rz(π/1024) ├──────────────────────────┤ X ├┤ Rz(-π/512) ├──────────────────────────┤ X ├┤ Rz(π/512) ├───────────────┤ X ├┤ Rz(-π/256) ├───────────────┤ X ├┤ Rz(π/256) ├─────────────────────┤ X ├┤ Rz(-π/128) ├─────────────────────┤ X ├┤ Rz(π/128) ├──────────┤ X ├┤ Rz(-π/64) ├──────────┤ X ├┤ Rz(π/64) ├────────────────┤ X ├┤ Rz(-π/32) ├────────────────┤ X ├┤ Rz(π/32) ├─────┤ X ├┤ Rz(-π/16) ├─────┤ X ├┤ Rz(π/16) ├───────────┤ X ├┤ Rz(-π/8) ├───────────┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐┌────────────┐                          ┌───┐┌───────────┐                ┌───┐└─┬─┘└┬────────────┤               ┌───┐└─┬─┘├───────────┬┘                     ┌───┐└─┬─┘├────────────┤                     ┌───┐└─┬─┘├───────────┤          ┌───┐└─┬─┘├───────────┬┘          ┌───┐└─┬─┘└┬──────────┤                ┌───┐└─┬─┘├───────────┬┘                ┌───┐└─┬─┘└┬──────────┤     ┌───┐└─┬─┘├───────────┤     ┌───┐└─┬─┘├──────────┤           ┌───┐└─┬─┘└┬──────────┤           ┌───┐└─┬─┘├─────────┬┘┌───┐└─┬─┘└┬──────────┤┌───┐└─┬─┘├─────────┬┘┌─────────┐└─┬─┘└──┬────┬──┘┌─────────┐└─┬─┘├─────────┤└─┬─┘└──────────┘└─┬─┘└─────────┘└─────────┘└────┘└─────────┘
q_1 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/512) ├──────────────────────────┤ X ├┤ Rz(π/512) ├────────────────┤ X ├─

In [ ]:
# grade your work!

grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Oops 😕! The quantum circuit returned by the supplied pass manager is not routed onto a line graph.
Please review your answer and try again.


In [ ]:

# Create a StagedPassManager
pm_staged = StagedPassManager()

# Initial unrolling of 3-qubit gates
pm_staged.init = generate_unroll_3q(None)

# Layout stage with trivial layout and embedding
pm_staged.layout = PassManager()
pm_staged.layout += TrivialLayout(cm)
pm_staged.layout += generate_embed_passmanager(cm)

# Routing stage with SabreSwap
pm_staged.routing = PassManager()
pm_staged.routing += SabreSwap(cm)

# Synthesis stage with Unroll3qTo1q2q
pm_staged.translation = PassManager()
pm_staged.translation += Unroll3qTo1q2q()

# Run the pass manager
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

ImportError: cannot import name 'Unroll3qTo1q2q' from 'qiskit.transpiler.passes' (/usr/local/lib/python3.10/dist-packages/qiskit/transpiler/passes/__init__.py)